In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("/kaggle/input/uci-turkiye-student-evaluation-data-set/turkiye-student-evaluation_generic.csv")

In [ ]:
train.head()

# DATA VISULIZATION

In [ ]:
sns.countplot(x="class",data=train)

Number of course taken by Student(course No. 3 is student favourite,Course No. 12 is mostly disliked by them)

In [ ]:
sns.countplot(x="class",hue="nb.repeat",data=train)

Course no. 13 is mostly repeated by students

In [ ]:
sns.countplot(x="nb.repeat",hue="difficulty",data=train)

Easy courses are also repeated by students

In [ ]:
sns.countplot(x="difficulty",hue="nb.repeat",data=train)

In [ ]:
sns.countplot(x="difficulty",hue="attendance",data=train)

In [ ]:
sns.countplot(x="class",hue="instr",data=train)

Instructor1:Course No.-2,7,10 Instructor2:Course No.-1,6,11,13 Instructor3:Course No.-3,4,5,8,9,12,13 Course No. 13 is jointly taken by Intructor1,Instructor2 Instructor1 is least popular May be

In [ ]:
sns.countplot(x="class",hue="difficulty",data=train)

In [ ]:
for i in train.columns:
    train.hist(i)
    plt.show()

# DATA PREPROCESSING

In [ ]:
X_res=train.values

In [ ]:
X_res

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
features=scaler.fit_transform(X_res)

In [ ]:
features

# CLUSTERING

In [ ]:
from sklearn.cluster import KMeans
k = 5
kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(features)

In [ ]:
kmeans.labels_

In [ ]:
kmeans.inertia_

Lets find suitable value for k using elbow method and silhoutte score

In [ ]:
m=[]
for k in range(1,80):
    km=KMeans(n_clusters=k, random_state=42).fit(features)
    m.append(km.inertia_)


In [ ]:
m

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(range(1, 80),m, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.show()

value of k will be between 0 to 10

In [ ]:
l=[]
p=[]
for k in range(1,10):
    km=KMeans(n_clusters=k, random_state=42).fit(features)
    l.append(km.inertia_)
    p.append(km)
plt.figure(figsize=(20,10))
plt.plot(range(1, 10),l, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.show()

value of k  may be 3 lets cross check using silhoutte score (l stores model inertia,p stores all model

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_scores =[]
for model in p[1:]:
    s=silhouette_score(features, model.labels_)
    silhouette_scores.append(s)
    

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(range(1, 9),silhouette_scores, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Silhouette score", fontsize=14)

plt.show()



Silhoutte curve is not as we want(their might be some error)

# May be we should try reducing dimensions before clustering 

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)


In [ ]:
f=pca.fit_transform(features)

In [ ]:
f

In [ ]:
l=[]
p=[]
for k in range(1,10):
    km=KMeans(n_clusters=k, random_state=42).fit(features)
    l.append(km.inertia_)
    p.append(km)
plt.figure(figsize=(20,10))
plt.plot(range(1, 10),l, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.show()

Elbow method again giving value of k to be 3 or 4

In [ ]:
silhouette_scores =[]
for model in p[1:]:
    s=silhouette_score(f, model.labels_)
    silhouette_scores.append(s)
plt.figure(figsize=(20, 10))
plt.plot(range(1, 9),silhouette_scores, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Silhouette score", fontsize=14)
plt.show()  

k=4 is the best value for number of cluster(Elbow method is giving k=3,4 silhoutte scores is giving k=4 so intersection will be k=4)

Clustering after PCA in this case produce better silhoutte scores graph

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42)
y_pred = kmeans.fit_predict(f)

In [ ]:
t=kmeans.labels_

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(f[:,0], f[:,1], c=kmeans.labels_.astype(float))
plt.show()

# CLASSIFICATION ALGORITHM(MULTICLASS CLASSIFICATION)

# RANDOM FOREST CLASSIFIER IS THE BEST ALGORITHM TO SOLVE MULTICLASS CLASSIFICATION PROBLEMS

In [ ]:
from sklearn.model_selection import train_test_split

np.random.seed(1234)

x_train,x_test,y_train,y_test = train_test_split(f,t, train_size=0.80, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
rnd_clf = RandomForestClassifier(n_estimators=200, max_leaf_nodes=8, random_state=42)
cv_scores = cross_val_score(rnd_clf, x_train, y_train, cv=10)
cv_scores.mean()

In [ ]:
rnd_clf.fit(x_train,y_train)
rnd_clf.score(x_test,y_test)

we are getting 99% accuracy

Lets try various n_estimators and max_leaf_nodes value

In [ ]:
n_estimator=[10,100,400,500,600,700,800,2000]
max_leaf_nodes=[2,4,6,8,16]
for i in n_estimator:
    for k in max_leaf_nodes:
       rnd_clf = RandomForestClassifier(n_estimators=i, max_leaf_nodes=k, random_state=42)
       rnd_clf.fit(x_train, y_train)
       t=rnd_clf.score(x_test,y_test)
       print(i,k,t)
    print("-"*40)

Our Random Forest Classifier is giving as 99% accuracy (min hyperparameter value for best score n_estimator=500,max_leaf_nodes=6)